In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('../../')

In [62]:
from src.generation import get_model, format_docs, QA_SYSTEM_PROMPT, QA_PROMPT, LLAMA_PROMPT_TEMPLATE
from src.indexing import get_multivector_retriever

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import chromadb

In [13]:
persistent_client = chromadb.PersistentClient(path='../../data/chroma')

In [14]:
save_path = 'D:\Ahmed\saudi-rag-project\data'

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20804\3935730633.py:1: SyntaxWarning: invalid escape sequence '\A'
  save_path = 'D:\Ahmed\saudi-rag-project\data'


In [15]:
PARENT_CHUNK_SIZE = 1200
PARENT_CHUNK_OVERLAP = 400

CHILD_CHUNK_SIZE = 300
CHILD_CHUNK_OVERLAP = 0

EMBEDDING_MODEL_NAME = 'intfloat/multilingual-e5-small'

COLLECTION_NAME = f"PARENT_{PARENT_CHUNK_SIZE}_{PARENT_CHUNK_OVERLAP}_CHILD_{CHILD_CHUNK_SIZE}_{CHILD_CHUNK_OVERLAP}_{EMBEDDING_MODEL_NAME}"
COLLECTION_NAME = COLLECTION_NAME.replace('/', '_').replace('-', '_')

retriever = get_multivector_retriever(persistent_client, EMBEDDING_MODEL_NAME, COLLECTION_NAME, save_path)

In [81]:
from langchain_core.prompts import PromptTemplate

QA_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Answer in Arabic only."""

LLAMA_PROMPT_TEMPLATE = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Answer:"""

qa_prompt_template = PromptTemplate.from_template(LLAMA_PROMPT_TEMPLATE.format(system_prompt=QA_SYSTEM_PROMPT, user_message=QA_PROMPT))

In [82]:
qa_prompt_template

PromptTemplate(input_variables=['context', 'question'], template="<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Answer in Arabic only.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: {question} \nContext: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAnswer:")

In [87]:
MODEL_NAME = "meta-llama/Llama-3-70b-chat-hf"
llm = get_model(MODEL_NAME)

In [88]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | qa_prompt_template
    | llm.bind(stop=["<|eot_id|>"])
    | StrOutputParser()
)

In [89]:
rag_chain.invoke("ما هو اجمالي صافي الربح بعد الزكاة للمجموعة عام 2021؟")

' صافي الربح بعد الزكاة للمجموعة عام 2021 هو 587.7 مليون ريال سعودي.'